# Stock Volatility 

#### David Montoto
#### Vatsal Bagri

About:

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch
from torch import nn
from sklearn.model_selection import train_test_split

### Dataset 1 - Stock data from Yahoo Finance

Our initial dataset, a collection of stocks along with their varying prices and dividends with the given data, are provided from Yahoo Finance. Most of the data is easy to work with as it has been polished. However, the 'Date' category can be simplified to just the provided date not including time. 

In [ ]:
df1_initial = pd.read_csv('data/stock_details_5_years.csv', header = 0)
df1_initial.head()

: 

In [13]:
df1_initial['Date'] = pd.to_datetime(df1_initial['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0])
df1_initial.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA


### Data 2 - Volatility Data For Each Company (Computed and Wrangled From Data set 1)

We will create a second dataset. This dataset will be taken from a subset of the date from out first created dataset. We aim to group different stocks by volatility. This data is grouped seperately because it will be used seperately from the rest of the data in dataset 1. 

In [17]:
df1_initial['perc_change'] = abs(((df1_initial['Close'] - df1_initial['Open']) / df1_initial['Open']) * 100.0)

changes_df = df1_initial.groupby('Company')['perc_change'].apply(list).reset_index()

changes_df['length'] = changes_df['perc_change'].apply(len)
full_length = changes_df['length'].max()
final_df = changes_df[changes_df['length'] == full_length].drop(columns=['length'])

final_df.head(10)


,Company,perc_change
0,A,"[0.4773218052326546, 1.0051671761984158, 1.827..."
1,AAPL,"[1.702617183838854, 0.9484672021502785, 0.1951..."
2,ABBV,"[1.011124035815082, 4.442714837058878, 0.69974..."
3,ABEV,"[0.7025810290951009, 0.2347359508779906, 0.689..."
5,ABT,"[1.1459365284516383, 0.1623194184859994, 0.026..."
6,ACGL,"[0.38910046417311844, 0.5975397952022214, 1.11..."
7,ACN,"[0.702792270799572, 1.6999443205598022, 0.5121..."
8,ADBE,"[1.1081327004284363, 0.2637588112448539, 2.090..."
9,ADI,"[0.5112321590414718, 2.4749178330342025, 0.182..."
10,ADM,"[0.4801423443221319, 0.08699633737629665, 0.67..."


In [ ]:
X = np.array(final_df['perc_change'].tolist())
pca = PCA(n_components=250) 
X_pca = pca.fit_transform(X)

kmeans = KMeans(n_clusters=11, random_state=24) # use of 11 explained in writing above
cluster_labels = kmeans.fit_predict(X_pca)

silh_score = silhouette_score(X_pca, cluster_labels)
print('silhouette score:', silh_score)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='hsv', alpha=0.6)
plt.title('Baseline k-means clustering')
plt.xlabel('principal component x direction')
plt.ylabel('principal component y direction')
plt.colorbar()
plt.show()

: 

### PCA Analysis


In [ ]:
X = np.array(final_df['perc_change'].tolist())
pca = PCA()
pca.fit(X)

# elbow method shown here
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Components')
plt.ylabel('Variance')
plt.title('Variance vs. total components')
plt.grid(True)
plt.show()


### Baseline K Means and GMM Models

In [ ]:
X = np.array(final_df['perc_change'].tolist())
pca = PCA(n_components=84) 
X_pca = pca.fit_transform(X)

kmeans = KMeans(n_clusters=11, random_state=24) # use of 11 explained in writing above
cluster_labels = kmeans.fit_predict(X_pca)

silh_score = silhouette_score(X_pca, cluster_labels)
print('silhouette score:', silh_score)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='hsv', alpha=0.6)
plt.title('Baseline k-means clustering')
plt.xlabel('principal component x direction')
plt.ylabel('principal component y direction')
plt.colorbar()
plt.show()

In [ ]:
gmm = GaussianMixture(n_components=11, random_state=24)  
cluster_labels = gmm.fit_predict(X_pca)

silh_score = silhouette_score(X_pca, cluster_labels)
print('silhouette score:', silh_score)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='hsv', alpha=0.6)
plt.title('Baseline GMM clustering')
plt.xlabel('principal component x direction')
plt.ylabel('principal component y direction')
plt.colorbar()
plt.show()

### Comparison of Cluster Assignment between the two techiniques 

In [ ]:
kmeans_labels = kmeans.predict(X_pca)
gmm_labels = gmm.predict(X_pca)

# Compare the cluster assignments
print("KMeans cluster assignments:", kmeans_labels[:10])
print("GMM cluster assignments:", gmm_labels[:10])

kmeans_silhouette = silhouette_score(X_pca, kmeans_labels)
gmm_silhouette = silhouette_score(X_pca, gmm_labels)

print(f"KMeans Silhouette Score: {kmeans_silhouette}")
print(f"GMM Silhouette Score: {gmm_silhouette}")

### Cross validation and optimization of Baseline GMM and K Means

In [ ]:
silhouette_scores = []

# corss validation here, note it is unseeded
for n_clusters in range(2, 19):
    kmeans = KMeans(n_clusters=n_clusters)
    cluster_labels = kmeans.fit_predict(X_pca)
    silh_score = silhouette_score(X_pca, cluster_labels)
    silhouette_scores += [silh_score]

final_cluster_num = np.argmax(silhouette_scores) + 2 # add 2 because corresponding index is left shifted

print("best # of clusters:", final_cluster_num)
print("Corresponding best cross validation silhouette score", silhouette_scores[final_cluster_num - 2])

#kmeans used new found best k param
kmeans = KMeans(n_clusters=final_cluster_num, random_state=24)
cluster_labels = kmeans.fit_predict(X_pca)

#saving labl results for later analysis in finl-df
final_df['k-means label'] = cluster_labels

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='coolwarm', alpha=0.5)
plt.title('Tuned k-means Lables')
plt.xlabel('Principal Component x direction')
plt.ylabel('Principal Component y direction')
plt.colorbar()
plt.show()

# silhoutte score from graphed kmeans clustering
final_silhouette_score = silhouette_score(X_pca, cluster_labels)
print("Silhouette score for graphed clustered:", final_silhouette_score)

In [ ]:
silhouette_scores = []

# cross validation here, note this part is unseeded
for n_components in range(2, 19):
    gmm = GaussianMixture(n_components=n_components)
    cluster_labels = gmm.fit_predict(X_pca)
    silh_score = silhouette_score(X_pca, cluster_labels)
    silhouette_scores.append(silh_score)

final_cluster_num = np.argmax(silhouette_scores) + 2

print("best # of clusters:", final_cluster_num)
print("Corresponding best cross validation silhouette score", silhouette_scores[final_cluster_num - 2])

gmm = GaussianMixture(n_components=final_cluster_num, random_state=24)
cluster_labels = gmm.fit_predict(X_pca)

# Update final_df with cluster labels
final_df['gmm labels'] = cluster_labels

# Plot the clustering labels

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='coolwarm', alpha=0.5)
plt.title('Tuned gmm Lables')
plt.xlabel('Principal Component x direction')
plt.ylabel('Principal Component y direction')
plt.colorbar()
plt.show()

final_silhouette_score = silhouette_score(X_pca, cluster_labels)
print("Silhouette score for graphed clustered:", final_silhouette_score)